# __Q1: Supposons que je lis le document numéro '422908' dans ma matrice. Appliquez l'algorithme Page rank pour déterminer les autres lectures recommandées. En plus de la simple recommandation des références de '422908', appliquez au moins une variation de cette approche de base, comme celle exposée en classe qui consiste à étendre le sous-ensemble S (références) à S' (références des références). Expliquez la démarche que vous avez prise.__

In [ ]:
import pandas as pd
import networkx as nx
from itertools import islice
import matplotlib.pyplot as plt


from jupyterthemes import jtplot

jtplot.style()

Le fichier `citeseet.csv` est simplement la `rtable` à l'adresse `http://cours.polymtl.ca/MDesmarais/log6308/Public/citeseer.rtable`. Le code ci-dessous permet de storer les informations dans un dataframe de Pandas.

In [ ]:
df = pd.read_csv("citeseer.csv", index_col=0)
df.columns = [int(col[1:]) for col in df.columns]
df.head()

La librarie `networkx` permet de prendre le dataframe et d'en faire un objet représentant un graphe. Cela permet de manipuler plus facilement les articles.

[Networkx](https://networkx.github.io/documentation/stable/)

In [ ]:
G = nx.from_pandas_adjacency(df, create_using=nx.DiGraph())
nx.info(G)

In [ ]:
G_no_isolates = G
G_no_isolates.remove_nodes_from(list(nx.isolates(G)))
fig = plt.figure(figsize=(30, 30))
fig.suptitle("graph without isolates", fontsize=20)
nx.draw(G_no_isolates, with_labels=True, font_weight='bold')
plt.show()

La librarie permet entre autres d'[appliquer l'algorithme de PageRank](https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank_numpy.html) sur un graphe. 
Une autre fonction possible dans la librarie permet d'appliquer [une variante](https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html) qui converti tous les arêtes en arête bidirectionnel, ignorant alors le sens des liens entre les articles.

In [ ]:
pr = nx.pagerank_numpy(G)
#print some to make sure they make sense
list(islice(pr.items(), 5))

## Fonction générale
La fonction `get_top_recommendations` ci-dessous ne comporte pas directement de calcul de pagerank. Elle ne fait que prendre un article, un graphe, et un dictionnaire `key=article value=pagerank_score` afin de redonner les recommandations selon le nombre de recommandations et le niveau d'ensembles voulus.

Par exemple, `max_deepness=1` indique que l'on veut se limiter à S et `max_deepness=2` indique que l'on veut également considérer les articles se trouvant dans l'ensemble S'. 

`graph.neighbors(n)` [retourne tous les noeuds m où il existe une arête de n qui pointe vers m](https://networkx.github.io/documentation/stable/reference/classes/generated/networkx.DiGraph.neighbors.html).

In [ ]:
def get_top_recommendations(page_ranks, graph, article, n_recommendations=10, max_deepness=1):
    possible_article_recommendations = []
    last_discovered_nodes = [article]
    deepness_reached = 0
    while deepness_reached <= max_deepness:
        new_discovered_nodes = []
        for discovered_node in last_discovered_nodes:
            for neighbor in G.neighbors(discovered_node):
                new_discovered_nodes.append(neighbor)
                if neighbor not in possible_article_recommendations and neighbor != article:
                    possible_article_recommendations.append(neighbor)
        last_discovered_nodes = new_discovered_nodes
        deepness_reached += 1
    pr_neighbors = {article_key:page_ranks[article_key] for article_key in possible_article_recommendations}
    return sorted(pr_neighbors, key=pr_neighbors.get, reverse=True)[:n_recommendations] 

## Variation à l'approche Pagerank de base
D'abord, nous allons tenter l'expérience avec S' au lieu de S seulement. 

De plus, nous allons tenter une autre approche, qui est de considérer un graphe non-orienté. Nous justifions l'essai de cette alternative en supposant que les articles peuvent être tout autant pertinents même s'il ne sont pas référencés par l'article de départ, en autant qu'ils référencient celui-ci. Autrement dit, selon cette approche, que ce soit n->m ou m->n, l'important est simplement qu'il y ait un lien entre ces articles, le sens n'a plus ou moins d'importance..

## Résultats
Les résultats suivants représentent les recommandations d'articles pour S et S' ainsi que S et S' pour une variante de l'algorithme de pagerank qui considère tous les arêtes du graphe comme bidirectionnelles. 

In [ ]:
document = 422908

In [ ]:
# S
print(get_top_recommendations(pr, G, document))

# S'
print(get_top_recommendations(pr, G, document, max_deepness=2))

On peut voir que les résultats sont identiques pour les 5 premières recommendations. Par contre, à la sixième recommendation, les deux approches divergent avec S' qui contient un article avec un score pagerank assez élevé pour faire partie des recommendations (61863) que S ne contient pas.

L'expérience qui suit présente une autre approche, qui est d'appliquer [pagerank en ignorant la direction des arêtes](https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html) entre les articles.

In [ ]:
G = nx.from_pandas_adjacency(df) # creates an undirected graph
pr = nx.pagerank(G)

# S
print(get_top_recommendations(pr, G, document))

# S'
print(get_top_recommendations(pr, G, document, max_deepness=2))

On peut voir qu'avec cette approche, les articles recommandés diffèrent de l'aproche de base après les deux premiers, et que de nouveaux articles font surface (38085, 245669, 164643, 547939, 220337, 500980, 371548, 368281, 83263).

## Discussion
TODO: discuter de la qualité des résultats (quand l'API du prof va marcher) 

# __Q2: Comparez les résultats obtenus avec une approche basée sur la similarité des articles dans un espace vectoriel, à l'instar du calcul de similarité de l'approche item-item. La mesure de la similarité et la façon de l'utiliser pour estimer la pertinence d'articles similaires est laissé à votre discrétion.__

# __Q3:  Utilisez une validation croisée pour évaluer la performance de l'approche item-item.__ Vous pouvez vous inspirer de l'approche utilisée dans l'article suivant pour la méthodologie à adopter:
McNee, S. M., Albert, I., Cosley, D., Gopalkrishnan, P., Lam, S. K., Rashid, A., Konstan, J. A., and Riedl, J. 2002. On the recommending of citations for research papers. In Proceedings of the 2002 ACM Conference on Computer Supported Cooperative Work (New Orleans, Louisiana, USA, November 16 - 20, 2002). CSCW '02. ACM, New York, NY, 116-125. DOI=http://doi.acm.org/10.1145/587078.587096
# Cependant, vous pouvez aussi adapter cette méthodologie ou en définir une différente. Par exemple, les "observations" que l'on peut vraisemblablement considérer comme valables pour les tests sont les références d'ordre 1 et 2 de la matrice d'adjacence et de sa transposée. Compte tenu du faible nombre de références, il est préférable d'adopter une approche "leave one out", où une seule "observation" est retirée du corpus à la fois, et où l'on compare une mesure du degré de confiance à prédire cette observation.